## Get embeddings
This notebook contains some helpful snippets you can use to embed text with the 'text-embedding-ada-002' model via Azure OpenAI API.

## Installation
Install the Azure Open AI SDK using the below command.

In [1]:
#r "nuget: Azure.AI.OpenAI, *-*"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.8

In [3]:
using Microsoft.DotNet.Interactive;

In [14]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");

// Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");

// Enter the deployment name you chose when you deployed the model.
var deployment = await Kernel.GetInputAsync("Provide deployment name");

### Import namesapaces and create an instance of `OpenAiClient` using the `azureOpenAIEndpoint` and the `azureOpenAIKey`

In [15]:
using Azure;
using Azure.AI.OpenAI;

In [16]:
OpenAIClient client = new (new Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey));

### 1. Load the dataset
The dataset used in this example is [fine-food reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews) from Amazon. The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of this dataset, consisting of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text).

We will combine the review summary and review text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

In [1]:
#!value --name dataSet --from-url https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/data/fine_food_reviews_1k.csv

### Create text embeddings using the `deployment`

In [17]:
foreach (var source in sources)
{
   var embeddings = await client.GetEmbeddingsAsync(deployment, new EmbeddingsOptions(source));
    embeddings.Value.Data[0].Display();
}

Embedding,"[ 0.0035413178, -0.004177728, -0.0042222086, -0.012919822, 0.0069594593, 0.012919822, -0.01044261, 0.0038081997, -0.020693613, -0.018709106, 0.01792899, 0.030492969, -0.001381456, 0.013008783, -0.0074521643, 0.025579605, 0.010832668, -0.018312205, -0.0061793425, 0.0043248557 ... (1516 more) ]"
Index,0


Embedding,"[ -0.00032866737, 0.011613192, -0.017493289, -0.025271054, 0.016691457, -0.0041661826, -0.011386006, -0.0013439029, -0.0010599209, -0.01840203, 0.009936028, 0.030015223, -0.006935842, 0.019805236, -0.006544949, 0.013023078, 0.024081672, -0.026607439, 0.0024789956, 0.0012904474 ... (1516 more) ]"
Index,0


Embedding,"[ -0.0069043823, 0.011252086, -0.009558876, -0.019589962, 0.0072855228, 0.01814635, -0.019158227, -0.007690274, 0.0005830948, -0.0025954675, 0.003659626, 0.019940747, -0.006442291, 0.009821964, 0.0091338875, 0.011157643, 0.021101033, -0.009484671, 0.01713447, -0.0015852759 ... (1516 more) ]"
Index,0


Embedding,"[ -0.006491534, 0.0074753026, -0.0072143027, -0.024065522, 0.011959146, 0.011490684, -0.01856445, -0.01699845, -0.00617365, 0.00081018696, 0.013719222, 0.015258451, -0.006123457, 0.0006077448, -0.0014789989, 0.019180141, 0.0208666, -0.02022414, 0.008586225, -0.00839884 ... (1516 more) ]"
Index,0


Embedding,"[ -0.00646148, 0.006511466, -0.017621614, -0.033403754, 0.0056850365, -0.00899742, -0.018354736, -0.018328078, -0.0118032815, 0.0015145657, 0.010796904, 0.009730543, -0.020567436, 0.0149957, -0.007831087, 0.021940375, 0.022433566, -0.026245806, 0.012256485, -0.009423964 ... (1516 more) ]"
Index,0


Embedding,"[ -0.021144673, 0.009870633, -0.0070905495, -0.0359005, 0.0052961926, 0.00646904, -0.022307497, -0.02115804, -0.017776495, 0.0027934492, 0.004243637, 0.010224826, -0.014956314, 0.018377956, -0.017081475, 0.021425355, 0.021545647, -0.027506787, 0.009736975, -0.008921662 ... (1516 more) ]"
Index,0


Embedding,"[ -0.006183072, 0.0080613885, -0.006550714, -0.038528893, 0.008175024, 0.007386264, -0.020855334, -0.005504605, -0.008195077, 0.005287362, 0.011443696, 0.0071991007, -0.015708344, 0.015039904, -0.017352708, 0.019117389, 0.019932887, -0.024692181, 0.012680311, -0.0037098431 ... (1516 more) ]"
Index,0


Embedding,"[ -0.0034153261, 0.004098391, -0.014244894, -0.020133844, 0.0004306461, -0.002992555, -0.019895103, -0.0057795276, -0.017202633, -4.517846E-05, 0.012116117, 0.016924102, -0.009967446, 0.012401281, -0.0034518004, 0.020054264, 0.014417318, -0.018820768, 0.006611806, -0.0014614611 ... (1516 more) ]"
Index,0


### Generate a single request with a batch of inputs, this is more efficient and saves api calls

In [18]:
var batchSize =  4;
var batchCount = sources.Count / batchSize;
for(var i = 0; i < batchCount; i++)
{
   var embeddings = await client.GetEmbeddingsAsync(deployment, new EmbeddingsOptions(sources.Skip(i*batchSize).Take(batchSize)));
    embeddings.Value.Data.Display();
}

index value 0 Azure.AI.OpenAI.EmbeddingItem Embedding [ 0.0035413178, -0.004177728, -0.0042222086, -0.012919822, 0.0069594593, 0.012919822, -0.01044261, 0.0038081997, -0.020693613, -0.018709106, 0.01792899, 0.030492969, -0.001381456, 0.013008783, -0.0074521643, 0.025579605, 0.010832668, -0.018312205, -0.0061793425, 0.0043248557 ... (1516 more) ] Index 0 1 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.00032866737, 0.011613192, -0.017493289, -0.025271054, 0.016691457, -0.0041661826, -0.011386006, -0.0013439029, -0.0010599209, -0.01840203, 0.009936028, 0.030015223, -0.006935842, 0.019805236, -0.006544949, 0.013023078, 0.024081672, -0.026607439, 0.0024789956, 0.0012904474 ... (1516 more) ] Index 1 2 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.0069043823, 0.011252086, -0.009558876, -0.019589962, 0.0072855228, 0.01814635, -0.019158227, -0.007690274, 0.0005830948, -0.0025954675, 0.003659626, 0.019940747, -0.006442291, 0.009821964, 0.0091338875, 0.011157643, 0.021101033, -0.009484671, 0.01713447, -0.0015852759 ... (1516 more) ] Index 2 3 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.006491534, 0.0074753026, -0.0072143027, -0.024065522, 0.011959146, 0.011490684, -0.01856445, -0.01699845, -0.00617365, 0.00081018696, 0.013719222, 0.015258451, -0.006123457, 0.0006077448, -0.0014789989, 0.019180141, 0.0208666, -0.02022414, 0.008586225, -0.00839884 ... (1516 more) ] Index 3

index value 0 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.00646148, 0.006511466, -0.017621614, -0.033403754, 0.0056850365, -0.00899742, -0.018354736, -0.018328078, -0.0118032815, 0.0015145657, 0.010796904, 0.009730543, -0.020567436, 0.0149957, -0.007831087, 0.021940375, 0.022433566, -0.026245806, 0.012256485, -0.009423964 ... (1516 more) ] Index 0 1 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.021144673, 0.009870633, -0.0070905495, -0.0359005, 0.0052961926, 0.00646904, -0.022307497, -0.02115804, -0.017776495, 0.0027934492, 0.004243637, 0.010224826, -0.014956314, 0.018377956, -0.017081475, 0.021425355, 0.021545647, -0.027506787, 0.009736975, -0.008921662 ... (1516 more) ] Index 1 2 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.006183072, 0.0080613885, -0.006550714, -0.038528893, 0.008175024, 0.007386264, -0.020855334, -0.005504605, -0.008195077, 0.005287362, 0.011443696, 0.0071991007, -0.015708344, 0.015039904, -0.017352708, 0.019117389, 0.019932887, -0.024692181, 0.012680311, -0.0037098431 ... (1516 more) ] Index 2 3 Azure.AI.OpenAI.EmbeddingItem Embedding [ -0.0034153261, 0.004098391, -0.014244894, -0.020133844, 0.0004306461, -0.002992555, -0.019895103, -0.0057795276, -0.017202633, -4.517846E-05, 0.012116117, 0.016924102, -0.009967446, 0.012401281, -0.0034518004, 0.020054264, 0.014417318, -0.018820768, 0.006611806, -0.0014614611 ... (1516 more) ] Index 3